In [128]:
import pandas as pd
import numpy as np

from selenium import webdriver

from datetime import datetime

import time

from requests import get
from fake_useragent import UserAgent

In [138]:
df = pd.read_csv('date_time_loc_teams.csv', parse_dates=['Date'])
df = df.loc[:, df.columns[1:]]
df

,Date,Time,Location,HomeTeam,AwayTeam
0,2024-04-25,20:00,EGKK,Brighton,Man City
1,2024-04-24,20:00,EGCC,Man United,Sheffield United
2,2024-04-24,20:00,EGGP,Everton,Liverpool
3,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle
4,2024-04-24,19:45,EGBB,Wolves,Bournemouth
...,...,...,...,...,...
2231,2018-08-11,15:00,EGNT,Newcastle,Tottenham
2232,2018-08-11,15:00,EGNM,Huddersfield,Chelsea
2233,2018-08-11,15:00,EGLC,Fulham,Crystal Palace
2234,2018-08-11,15:00,EGHH,Bournemouth,Cardiff


In [139]:
teams = df['HomeTeam'].value_counts().index
teams

Index(['Newcastle', 'West Ham', 'Everton', 'Crystal Palace', 'Wolves',
       'Arsenal', 'Man City', 'Liverpool', 'Tottenham', 'Man United',
       'Brighton', 'Chelsea', 'Leicester', 'Southampton', 'Burnley',
       'Aston Villa', 'Bournemouth', 'Fulham', 'Leeds', 'Watford',
       'Sheffield United', 'Brentford', 'Norwich', 'Nott'm Forest',
       'West Brom', 'Huddersfield', 'Cardiff', 'Luton'],
      dtype='object', name='HomeTeam')

In [140]:
# есть API для выкачивания всех elo rating по командам

# Я начинаю вручную скачивать .csv файлы с API http://clubelo.com/API
# Чтобы легче вбивать в поиск, делаю все поисковые запросы здесь:

In [212]:
# for team in teams:
#     try:
#         df_find = pd.read_csv(f'{"".join(team.split())}.csv')
#     except:
#         print(f'{team} Потерялась!')
        
# # Не было файла только по одной команде. Тк она участвует в небольшом количестве рассматриваемых сезонов, решено
# # было не учитывать матчи с ней

In [142]:
# for team in teams:
#     print(f'api.clubelo.com/{"".join(team.split())}')

In [143]:
teams_to_concat = []

for team in teams:
    if team != "Nott'm Forest":
        df_to_add = pd.read_csv(f'{"".join(team.split())}.csv')
        df_to_add = df_to_add.loc[(df_to_add['From'] >= '2018-08-01') & (df_to_add['To'] <= '2024-04-30')]
        teams_to_concat.append(df_to_add.loc[:, ['Club', 'Elo', 'From', 'To']])

In [144]:
df_all_teams = pd.concat(teams_to_concat)
df_all_teams

,Club,Elo,From,To
6061,Newcastle,1670.470215,2018-08-03,2018-08-04
6062,Newcastle,1670.470215,2018-08-05,2018-08-08
6063,Newcastle,1670.470215,2018-08-09,2018-08-09
6064,Newcastle,1670.871338,2018-08-10,2018-08-11
6065,Newcastle,1666.971924,2018-08-12,2018-08-14
...,...,...,...,...
3331,Luton,1608.824951,2024-04-12,2024-04-13
3332,Luton,1607.486450,2024-04-14,2024-04-17
3333,Luton,1603.200806,2024-04-18,2024-04-18
3334,Luton,1601.028809,2024-04-19,2024-04-20


In [209]:
# делаем дф для ноттмфорест

df_nottm

In [ ]:
pd.concat([df_all_teams, ])

In [145]:
# теперь надо сделать мердж по всем командам и играм

In [213]:
merged_home = df.merge(df_all_teams, left_on='HomeTeam', right_on='Club', how='left')
merged_home

,Date,Time,Location,HomeTeam,AwayTeam,Club,Elo,From,To
0,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1632.467285,2018-08-03,2018-08-05
1,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1632.467285,2018-08-06,2018-08-09
2,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1632.868408,2018-08-10,2018-08-11
3,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1623.589966,2018-08-12,2018-08-12
4,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1623.589966,2018-08-13,2018-08-16
...,...,...,...,...,...,...,...,...,...
1394015,2018-08-10,15:00,EGCC,Man United,Leicester,Man United,1795.381958,2024-04-18,2024-04-18
1394016,2018-08-10,15:00,EGCC,Man United,Leicester,Man United,1793.209839,2024-04-19,2024-04-22
1394017,2018-08-10,15:00,EGCC,Man United,Leicester,Man United,1793.209839,2024-04-23,2024-04-24
1394018,2018-08-10,15:00,EGCC,Man United,Leicester,Man United,1796.192505,2024-04-25,2024-04-25


In [214]:
merged_home = merged_home[(merged_home.Date >= merged_home.From) & (merged_home.Date <= merged_home.To)]
#merged_home.groupby(['Date', 'Club'])['Elo'].agg(['max'])
merged_home

,Date,Time,Location,HomeTeam,AwayTeam,Club,Elo,From,To
708,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1748.398682,2024-04-19,2024-04-25
1276,2024-04-24,20:00,EGCC,Man United,Sheffield United,Man United,1793.209839,2024-04-23,2024-04-24
1952,2024-04-24,20:00,EGGP,Everton,Liverpool,Everton,1692.962402,2024-04-23,2024-04-24
2637,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle,Crystal Palace,1708.831055,2024-04-22,2024-04-24
3303,2024-04-24,19:45,EGBB,Wolves,Bournemouth,Wolves,1711.067993,2024-04-21,2024-04-24
...,...,...,...,...,...,...,...,...,...
1390834,2018-08-11,15:00,EGNT,Newcastle,Tottenham,Newcastle,1670.871338,2018-08-10,2018-08-11
1391516,2018-08-11,15:00,EGNM,Huddersfield,Chelsea,Huddersfield,1567.101318,2018-08-10,2018-08-11
1392003,2018-08-11,15:00,EGLC,Fulham,Crystal Palace,Fulham,1633.799683,2018-08-10,2018-08-11
1392670,2018-08-11,15:00,EGHH,Bournemouth,Cardiff,Bournemouth,1673.780518,2018-08-10,2018-08-11


In [215]:
merged_away = df.merge(df_all_teams, left_on='AwayTeam', right_on='Club', how='left')
merged_away

,Date,Time,Location,HomeTeam,AwayTeam,Club,Elo,From,To
0,2024-04-25,20:00,EGKK,Brighton,Man City,Man City,1974.560547,2018-08-03,2018-08-09
1,2024-04-25,20:00,EGKK,Brighton,Man City,Man City,1974.961670,2018-08-10,2018-08-12
2,2024-04-25,20:00,EGKK,Brighton,Man City,Man City,1982.867065,2018-08-13,2018-08-16
3,2024-04-25,20:00,EGKK,Brighton,Man City,Man City,1985.608765,2018-08-17,2018-08-19
4,2024-04-25,20:00,EGKK,Brighton,Man City,Man City,1987.186157,2018-08-20,2018-08-23
...,...,...,...,...,...,...,...,...,...
1394131,2018-08-10,15:00,EGCC,Man United,Leicester,Leicester,1653.900269,2024-04-21,2024-04-22
1394132,2018-08-10,15:00,EGCC,Man United,Leicester,Leicester,1653.900269,2024-04-23,2024-04-23
1394133,2018-08-10,15:00,EGCC,Man United,Leicester,Leicester,1666.075806,2024-04-24,2024-04-24
1394134,2018-08-10,15:00,EGCC,Man United,Leicester,Leicester,1666.075806,2024-04-25,2024-04-27


In [216]:
merged_away = merged_away[(merged_away.Date >= merged_away.From) & (merged_away.Date <= merged_away.To)]
merged_away

,Date,Time,Location,HomeTeam,AwayTeam,Club,Elo,From,To
477,2024-04-25,20:00,EGKK,Brighton,Man City,Man City,2038.480591,2024-04-19,2024-04-25
1128,2024-04-24,20:00,EGCC,Man United,Sheffield United,Sheffield United,1551.766602,2024-04-21,2024-04-24
1630,2024-04-24,20:00,EGGP,Everton,Liverpool,Liverpool,1923.581787,2024-04-22,2024-04-24
2314,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle,Newcastle,1811.576782,2024-04-19,2024-04-24
3096,2024-04-24,19:45,EGBB,Wolves,Bournemouth,Bournemouth,1695.033447,2024-04-22,2024-04-24
...,...,...,...,...,...,...,...,...,...
1391187,2018-08-11,15:00,EGNT,Newcastle,Tottenham,Tottenham,1914.848877,2018-08-10,2018-08-11
1391760,2018-08-11,15:00,EGNM,Huddersfield,Chelsea,Chelsea,1837.004272,2018-08-10,2018-08-11
1392302,2018-08-11,15:00,EGLC,Fulham,Crystal Palace,Crystal Palace,1692.951660,2018-08-10,2018-08-11
1392986,2018-08-11,15:00,EGHH,Bournemouth,Cardiff,Cardiff,1576.490356,2018-08-10,2018-08-11


In [240]:
merged_all = merged_home.merge(merged_away, on=['Date', 'HomeTeam', 'AwayTeam'], how='inner')
merged_all

,Date,Time_x,Location_x,HomeTeam,AwayTeam,Club_x,Elo_x,From_x,To_x,Time_y,Location_y,Club_y,Elo_y,From_y,To_y
0,2024-04-25,20:00,EGKK,Brighton,Man City,Brighton,1748.398682,2024-04-19,2024-04-25,20:00,EGKK,Man City,2038.480591,2024-04-19,2024-04-25
1,2024-04-24,20:00,EGCC,Man United,Sheffield United,Man United,1793.209839,2024-04-23,2024-04-24,20:00,EGCC,Sheffield United,1551.766602,2024-04-21,2024-04-24
2,2024-04-24,20:00,EGGP,Everton,Liverpool,Everton,1692.962402,2024-04-23,2024-04-24,20:00,EGGP,Liverpool,1923.581787,2024-04-22,2024-04-24
3,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle,Crystal Palace,1708.831055,2024-04-22,2024-04-24,20:00,EGLC,Newcastle,1811.576782,2024-04-19,2024-04-24
4,2024-04-24,19:45,EGBB,Wolves,Bournemouth,Wolves,1711.067993,2024-04-21,2024-04-24,19:45,EGBB,Bournemouth,1695.033447,2024-04-22,2024-04-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2018-08-11,15:00,EGNT,Newcastle,Tottenham,Newcastle,1670.871338,2018-08-10,2018-08-11,15:00,EGNT,Tottenham,1914.848877,2018-08-10,2018-08-11
2160,2018-08-11,15:00,EGNM,Huddersfield,Chelsea,Huddersfield,1567.101318,2018-08-10,2018-08-11,15:00,EGNM,Chelsea,1837.004272,2018-08-10,2018-08-11
2161,2018-08-11,15:00,EGLC,Fulham,Crystal Palace,Fulham,1633.799683,2018-08-10,2018-08-11,15:00,EGLC,Crystal Palace,1692.951660,2018-08-10,2018-08-11
2162,2018-08-11,15:00,EGHH,Bournemouth,Cardiff,Bournemouth,1673.780518,2018-08-10,2018-08-11,15:00,EGHH,Cardiff,1576.490356,2018-08-10,2018-08-11


In [241]:
merged_all = merged_all.loc[:, ['Date', 'Time_x', 'Location_x', 'HomeTeam', 'AwayTeam', 'Elo_x', 'Elo_y']]

merged_all.rename(columns={'Time_x': 'Time',
                  'Location_x': 'Location',
                  'Elo_x': 'Elo_HomeTeam',
                  'Elo_y': 'Elo_AwayTeam'}, inplace=True)

In [242]:
merged_all

,Date,Time,Location,HomeTeam,AwayTeam,Elo_HomeTeam,Elo_AwayTeam
0,2024-04-25,20:00,EGKK,Brighton,Man City,1748.398682,2038.480591
1,2024-04-24,20:00,EGCC,Man United,Sheffield United,1793.209839,1551.766602
2,2024-04-24,20:00,EGGP,Everton,Liverpool,1692.962402,1923.581787
3,2024-04-24,20:00,EGLC,Crystal Palace,Newcastle,1708.831055,1811.576782
4,2024-04-24,19:45,EGBB,Wolves,Bournemouth,1711.067993,1695.033447
...,...,...,...,...,...,...,...
2159,2018-08-11,15:00,EGNT,Newcastle,Tottenham,1670.871338,1914.848877
2160,2018-08-11,15:00,EGNM,Huddersfield,Chelsea,1567.101318,1837.004272
2161,2018-08-11,15:00,EGLC,Fulham,Crystal Palace,1633.799683,1692.951660
2162,2018-08-11,15:00,EGHH,Bournemouth,Cardiff,1673.780518,1576.490356


# Сверху финальный файл: снизу выгружаю в .csv

In [245]:
merged_all.to_csv('elo_ratings_all_teams')